In [1]:
qwen_model = '/kaggle/input/qwen7b/qwen_model'

In [2]:
!pip install -q unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 7.1 MB/s eta 0:00:00


In [3]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import os

# Load the model and tokenizer
model_path = "/kaggle/input/qwen7b/qwen_model/"

# Load the model
model, tokenizer = FastVisionModel.from_pretrained(
    model_path,
    load_in_4bit=True,
    use_gradient_checkpointing="unsloth",
)

# Enable gradient checkpointing
model.gradient_checkpointing_enable()

# Verify the model
print(type(model))  # Should output <class 'unsloth.FastVisionModel'>
print(model)  # Inspect the model object

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.6: Fast Qwen2_Vl vision patching. Transformers: 4.48.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.36G [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

<class 'peft.peft_model.PeftModelForCausalLM'>
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2VLForConditionalGeneration(
      (visual): Qwen2VisionTransformerPretrainedModel(
        (patch_embed): PatchEmbed(
          (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
        (rotary_pos_emb): VisionRotaryEmbedding()
        (blocks): ModuleList(
          (0-12): 13 x Qwen2VLVisionBlock(
            (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
            (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
            (attn): VisionSdpaAttention(
              (qkv): lora.Linear(
                (base_layer): Linear(in_features=1280, out_features=3840, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1280, out_features=16, bias=False)
         

In [4]:
from transformers import TextStreamer
from PIL import Image
import torch

# Enable the model for inference
model = FastVisionModel.for_inference(model)

# Load the image as a PIL image
image_path = "/kaggle/input/videoframedata/frame_data/frames/BHAIRAB DANCE IN LONDON--IamAbatar--UCfj7fHQkQ5X5ogj8hpXwmJg---4Bkky5pgJg_segment1_clip_2/frame_3.jpg"
image = Image.open(image_path).resize((224, 224))  # Resize the image to 224x224

# Prepare instruction and messages
instruction = """Answer the question based on the content of this image.."""
question = "Which dance is being performed in the video?"

# Prepare the messages in the required format
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": instruction},
            {"type": "image", "image": image},  # Pass the resized PIL image directly
            {"type": "text", "text": question}
        ]
    }
]

# Apply the chat template and tokenize the input
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")


# Define the text streamer
text_streamer = TextStreamer(
    tokenizer=tokenizer,
    skip_prompt=True  # Skip showing the input prompt in the output
)
print(text_streamer)

# Generate the output
with torch.no_grad():  # Disable gradient computation during inference
    _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128,
                       use_cache=True, temperature=1.5, min_p=0.1)

The dance being performed in the video is the Bhairab Dance, which is a traditional Newari dance from Nepal.<|im_end|>


In [5]:
# Load the image as a PIL image
image_path = "/kaggle/input/videoframedata/frame_data/frames/Kumari Dance_Charya Nritya_______ _____ 2076_Aloyana dongol_UCB7v02qvayFmzxiddLufw6w_trn6IKdHA_E_segment1_clip_2/frame_3.jpg"
image = Image.open(image_path).resize((224, 224))  # Resize the image to 224x224

# Prepare instruction and messages
instruction = """
Answer the question based on the content of this image. 
"""
question = "What is the vibe of the video?"

# Prepare the messages in the required format
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": instruction},
            {"type": "image", "image": image},  # Pass the resized PIL image directly
            {"type": "text", "text": question}
        ]
    }
]

# Apply the chat template and tokenize the input
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")


# Define the text streamer
text_streamer = TextStreamer(
    tokenizer=tokenizer,
    skip_prompt=True  # Skip showing the input prompt in the output
)
print(text_streamer)

# Generate the output
with torch.no_grad():  # Disable gradient computation during inference
    _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128,
                       use_cache=True, temperature=1.5, min_p=0.1)

The vibe of the video is vibrant and celebratory, reflecting the joyous spirit of the Bhairab dance, which is performed during the Bhairab Jatra festival.<|im_end|>


In [6]:
# Load the image as a PIL image
image_path = '/kaggle/input/videoframedata/frame_data/frames/-DHIME DANCE AND MUSIC _ Culture Of Newar--Sam Audio Video--S4PcERz4Eio_segment3_clip_1/frame_2.jpg'
image = Image.open(image_path).resize((224, 224))  # Resize the image to 224x224

# Prepare instruction and messages
instruction = """
Answer the question based on the content of this image. 

"""
question = "How many dancers are in the video?"

# Prepare the messages in the required format
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": instruction},
            {"type": "image", "image": image},  # Pass the resized PIL image directly
            {"type": "text", "text": question}
        ]
    }
]

# Apply the chat template and tokenize the input
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")


# Define the text streamer
text_streamer = TextStreamer(
    tokenizer=tokenizer,
    skip_prompt=True  # Skip showing the input prompt in the output
)
print(text_streamer)

# Generate the output
with torch.no_grad():  # Disable gradient computation during inference
    _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128,
                       use_cache=True, temperature=1.5, min_p=0.1)

The video features two dancers, a man and a woman, performing the traditional Tibetan dance known as the "Chyabrung."<|im_end|>


In [7]:
# Load the image as a PIL image
image_path = '/kaggle/input/videoframedata/frame_data/frames/-_ Newari Dhime baja dance performance at Nyatapola Temple_ Bhaktapur _ Yatra Nepal _ 2080_12_26 _--YATRA NEPAL--vjvXAPeAoYc_segment2_clip_1/frame_4.jpg'
image = Image.open(image_path).resize((224, 224))  # Resize the image to 224x224

# Prepare instruction and messages
instruction = """
Answer the question based on the content of this image. 

"""
question = "What kind of instruments are being used in this video?"

# Prepare the messages in the required format
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": instruction},
            {"type": "image", "image": image},  # Pass the resized PIL image directly
            {"type": "text", "text": question}
        ]
    }
]

# Apply the chat template and tokenize the input
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")


# Define the text streamer
text_streamer = TextStreamer(
    tokenizer=tokenizer,
    skip_prompt=True  # Skip showing the input prompt in the output
)
print(text_streamer)

# Generate the output
with torch.no_grad():  # Disable gradient computation during inference
    _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128,
                       use_cache=True, temperature=1.5, min_p=0.1)

The instruments being played in the video include traditional Nepalese drums, known as "dhol," which are played in a rhythmic pattern to accompany the dance.<|im_end|>


In [8]:
# Load the image as a PIL image
image_path = '/kaggle/input/videoframedata/frame_data/frames/-_ Newari Dhime baja dance performance at Nyatapola Temple_ Bhaktapur _ Yatra Nepal _ 2080_12_26 _--YATRA NEPAL--vjvXAPeAoYc_segment2_clip_1/frame_4.jpg'
image = Image.open(image_path).resize((224, 224))  # Resize the image to 224x224

# Prepare instruction and messages
instruction = """
Answer the question based on the content of this image. 

"""
question = "Where is the setting of this video?"

# Prepare the messages in the required format
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": instruction},
            {"type": "image", "image": image},  # Pass the resized PIL image directly
            {"type": "text", "text": question}
        ]
    }
]

# Apply the chat template and tokenize the input
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")


# Define the text streamer
text_streamer = TextStreamer(
    tokenizer=tokenizer,
    skip_prompt=True  # Skip showing the input prompt in the output
)
print(text_streamer)

# Generate the output
with torch.no_grad():  # Disable gradient computation during inference
    _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128,
                       use_cache=True, temperature=1.5, min_p=0.1)

The setting of this video is the Durbar Square in Kathmandu, Nepal, featuring the vibrant Newari culture and traditional Newari dances, specifically the Bhairab dance, which is performed during the Indra Jatra festival.<|im_end|>


In [9]:
# Load the image as a PIL image
image_path = '/kaggle/input/videoframedata/frame_data/frames/BHAIRAB DANCE IN LONDON--IamAbatar--UCfj7fHQkQ5X5ogj8hpXwmJg---4Bkky5pgJg_segment2_clip_2/frame_4.jpg'
image = Image.open(image_path).resize((224, 224))  # Resize the image to 224x224

# Prepare instruction and messages
instruction = """
Answer the question based on the content of this image. 

"""
question = "Who are these masked deties in video?"

# Prepare the messages in the required format
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": instruction},
            {"type": "image", "image": image},  # Pass the resized PIL image directly
            {"type": "text", "text": question}
        ]
    }
]

# Apply the chat template and tokenize the input
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")


# Define the text streamer
text_streamer = TextStreamer(
    tokenizer=tokenizer,
    skip_prompt=True  # Skip showing the input prompt in the output
)
print(text_streamer)

# Generate the output
with torch.no_grad():  # Disable gradient computation during inference
    _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128,
                       use_cache=True, temperature=1.5, min_p=0.1)

The masked deities in the video are likely representations of Bhairab, a powerful deity associated with the Hindu pantheon, often depicted with a fierce appearance to symbolize protection and destruction. These masks are part of the traditional Bhairab dance, which is performed during the Bhairab Jatra festival in Nepal, showcasing the deity's strength and the community's reverence for him.<|im_end|>


In [10]:
# Load the image as a PIL image
image_path = '/kaggle/input/videoframedata/frame_data/frames/Indra Jatra 2074 _Halchok Bhairav Dance_--Sujit Sind--UC7ObPu4bVxKbJxBdFsnzqSg--7SI3qNlocBw_cut_clip_2/frame_4.jpg'
image = Image.open(image_path).resize((224, 224))  # Resize the image to 224x224

# Prepare instruction and messages
instruction = """
Answer the question based on the content of this image. 

"""
question = "How many masked deties are in the video and what do they represnet?"

# Prepare the messages in the required format
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": instruction},
            {"type": "image", "image": image},  # Pass the resized PIL image directly
            {"type": "text", "text": question}
        ]
    }
]

# Apply the chat template and tokenize the input
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")


# Define the text streamer
text_streamer = TextStreamer(
    tokenizer=tokenizer,
    skip_prompt=True  # Skip showing the input prompt in the output
)
print(text_streamer)

# Generate the output
with torch.no_grad():  # Disable gradient computation during inference
    _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128,
                       use_cache=True, temperature=1.5, min_p=0.1)

The video features two masked deities, representing the Bhairab and Bhairabini, who are central figures in the Newari culture, symbolizing power and protection.<|im_end|>


In [11]:
# Load the image as a PIL image
image_path = '/kaggle/input/videoframedata/frame_data/frames/KUMARI_TheJoshiramesh_UCOg58aroiIixHGkavfujkzA_Gg_9ly6zNOc_segment2_clip_2/frame_2.jpg'
image = Image.open(image_path).resize((224, 224))  # Resize the image to 224x224

# Prepare instruction and messages
instruction = """
Answer the question based on the content of this image. 

"""
question = "Is Kumari dance being performed on the video?"

# Prepare the messages in the required format
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": instruction},
            {"type": "image", "image": image},  # Pass the resized PIL image directly
            {"type": "text", "text": question}
        ]
    }
]

# Apply the chat template and tokenize the input
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")


# Define the text streamer
text_streamer = TextStreamer(
    tokenizer=tokenizer,
    skip_prompt=True  # Skip showing the input prompt in the output
)
print(text_streamer)

# Generate the output
with torch.no_grad():  # Disable gradient computation during inference
    _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128,
                       use_cache=True, temperature=1.5, min_p=0.1)

Yes, the Kumari dance is being performed in the video, showcasing the traditional attire and elaborate costumes of the Kumari dancers.<|im_end|>
